In [15]:
# 불균형 데이터 생성(1:9)   악성 : 양성
from typing import Concatenate
from sklearn.datasets import load_breast_cancer
import numpy as np
data = load_breast_cancer()
X, y = data.data, data.target
# X : (샘플 수, 특성 수) 형태의 2차원 배열 (569 x 30)
# y : 각 샘플의 정답 레이블(0=악성, 1=양성)인 1차원 배열 (길이 569)

# 악성을 소수 클래스로 생성
# 현재 클래스 분포 확인 
# np.unique(y, return_counts=True)
#  - y 안에 존재하는 고유값을 오름차순으로 반환(여기선 [0, 1])
#  - 각 값이 몇 개 있는지도 같이 반환
print(f'악성 양성의 오리지널 비율 : {np.unique(y, return_counts = True)}')
# 예) [0 1] / [212 357]  → 악성 212, 양성 357
# 클래스별 인덱스 뽑기
# np.where(y == 0)[0]  : y 값이 0(악성)인 위치의 인덱스들
# np.where(조건) 은 단순히 인덱스만 주는 게 아니라, 튜플(tuple) 형태로 결과를 반환
# | 코드                    | 반환 타입   | 설명                                 |
# | ---------------------  | ------- | ---------------------------------- |
# | `np.where(y == 0)`     | tuple   | `(array([...]),)` 형태               |
# | `np.where(y == 0)[0]`  | ndarray | `[인덱스, 인덱스, ...]` 형태 (실제 우리가 쓰는 것) |

m_index = np.where(y==0)[0]    # index 반환 악성
b_index = np.where(y==1)[0]

# 악성은 일부만, 양성은 더 많이 사용
# 악성의 30%만 사용, 양성은 전체  1:9
size_30 = int(len(m_index)*0.19)
selected_m_index = np.random.choice(m_index, size = size_30, replace = False)
selected_b_index = b_index
len(selected_m_index) + len(selected_b_index)

concatenate_selected_index = np.concatenate([selected_m_index, selected_b_index])
np.random.shuffle(concatenate_selected_index)

X_imb = X[concatenate_selected_index]
y_imb = y[concatenate_selected_index]

# 클래스 분포 확인
unique, counts = np.unique(y_imb, return_counts=True)
print('클래스 분포 : ')
for label, count in zip(unique, counts):
    percentage = count / len(y_imb) * 100
    print(f'클래스 {label} : {count}개 ({percentage : .1f}%)')

# 불균형인 상태로 진행
# 스케일링 정규화 StandardScaler
# LogisticRegression
# pipe
# 평가는 class report

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline, make_pipeline

X_train, X_test, y_train, y_test = train_test_split(X_imb, y_imb, test_size = 0.2, random_state = 42, stratify=y_imb)
print('1. 기본 모델 (불균형 무시)')
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(random_state = 42, max_iter=1000))
])
pipe.fit(X_train, y_train)
print(classification_report(y_test, pipe.predict(X_test)))


악성 양성의 오리지널 비율 : (array([0, 1]), array([212, 357]))
클래스 분포 : 
클래스 0 : 40개 ( 10.1%)
클래스 1 : 357개 ( 89.9%)
1. 기본 모델 (불균형 무시)
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        72

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80



In [16]:
print('불균형 해결 : 클래스 가중치 사용')
# 기존파이프라인의 clf 이름의 객체의 파라메터를 조정
from copy import deepcopy
pipe_weight = deepcopy(pipe)
pipe_weight.set_params(clf__class_weight = 'balanced')

# pipe_weight = Pipeline([
#     ('scaler',StandardScaler()),
#     ('clf', LogisticRegression(random_state=10,max_iter=1000,class_weight = 'balanced'))
# ])


pipe_weight.fit(X_train,y_train)
print( classification_report(y_test, pipe_weight.predict(X_test)  )   )

print("가중치 계산")
n_samples = len(y_train)
n_classes = 2
class_counts = np.bincount(y_train)
for i in range(n_classes):
    weight = n_samples / (n_classes * class_counts[i])
    print(f' 클래스 :{i} : {weight:.3f}')   


불균형 해결 : 클래스 가중치 사용
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        72

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80

가중치 계산
 클래스 :0 : 4.953
 클래스 :1 : 0.556


In [17]:
from sklearn.ensemble import RandomForestClassifier
print(' 불균형 : RandomFores (균형모드)')
pipe_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier(class_weight='balanced',random_state=42))
])
pipe_rf.fit(X_train, y_train)
y_pred = pipe_rf.predict(X_test)
print( classification_report(y_test, y_pred) )

 불균형 : RandomFores (균형모드)
              precision    recall  f1-score   support

           0       1.00      0.88      0.93         8
           1       0.99      1.00      0.99        72

    accuracy                           0.99        80
   macro avg       0.99      0.94      0.96        80
weighted avg       0.99      0.99      0.99        80

